In [ ]:
import numpy as np

In [ ]:
def impute(self, X, M):
        # Run EKF+RTS and impute missing xt as C mu_t|T
        ekf_out = self.ekf_forward(X, M)
        smooth_out = self.rts_smoother(ekf_out)
        mu_s = smooth_out['mu_smooth']  # T x K
        Ximp = X.copy().astype(float)
        for t in range(X.shape[0]):
            missing_idx = np.where(M[t] == 1)[0]
            if missing_idx.size > 0:
                Ximp[t, missing_idx] = (self.C @ mu_s[t])[missing_idx]
        return Ximp, smooth_out
    
def forecast_after_blackout(self, X, M, blackout_end_t, horizons=[1,3,6]):
    # Run EKF up to b (filter only), then do k-step predictions using A,Q,C,R
    T, D = X.shape
    ekf_out = self.ekf_forward(X[:blackout_end_t+1], M[:blackout_end_t+1])
    mu_b = ekf_out['mu_filt'][-1]
    Sigma_b = ekf_out['Sigma_filt'][-1]
    results = {}
    mu_pred = mu_b.copy()
    Sigma_pred = Sigma_b.copy()
    for k in range(1, max(horizons)+1):
        # propagate
        mu_pred = self.A @ mu_pred
        Sigma_pred = self.A @ Sigma_pred @ self.A.T + self.Q
        if k in horizons:
            mean_x = self.C @ mu_pred
            cov_x = self.C @ Sigma_pred @ self.C.T + self.R
            results[k] = {'mean': mean_x, 'cov': cov_x}
    return results
